In [4]:
import torch
from pytorch3d.renderer import (
    AlphaCompositor,
    RasterizationSettings,
    MeshRenderer,
    MeshRasterizer,
    PointsRasterizationSettings,
    PointsRenderer,
    PointsRasterizer,
    HardPhongShader,
)
from pytorch3d.io import load_obj
import pytorch3d
import matplotlib.pyplot as plt
import imageio
from PIL import Image
import numpy as np
from PIL import Image, ImageDraw
from tqdm.auto import tqdm
import mcubes
import pickle

def get_mesh_renderer(image_size=512, lights=None, device=None):
    """
    Returns a Pytorch3D Mesh Renderer.

    Args:
        image_size (int): The rendered image size.
        lights: A default Pytorch3D lights object.
        device (torch.device): The torch device to use (CPU or GPU). If not specified,
            will automatically use GPU if available, otherwise CPU.
    """
    if device is None:
        if torch.cuda.is_available():
            device = torch.device("cuda:0")
            
        else:
            device = torch.device("cpu")
            
    raster_settings = RasterizationSettings(
        image_size=image_size, blur_radius=0.0, faces_per_pixel=1,
    )
    renderer = MeshRenderer(
        rasterizer=MeshRasterizer(raster_settings=raster_settings),
        shader=HardPhongShader(device=device, lights=lights),
    )
    return renderer

In [80]:
class MyCowRenderer:
    def __init__(self, path="data/cow.obj",image_size=512, 
                 color=[0.7, 0.7, 1],device=None):
        self.path = path
        self.image_size = image_size
        self.color = color
        if device is None:
            if torch.cuda.is_available():
                self.device = torch.device("cuda:0")
                print("Using CUDA")
            else:
                self.device = torch.device("cpu")
                print("Using CPU")
        self.renderer = self.get_mesh_renderer(image_size=self.image_size,device=self.device)
        self.lights = pytorch3d.renderer.PointLights(location=[[0, 0, -3]], device=self.device)
        self.cameras = None
        self.mesh = None
        self.textured_mesh=None
        self.load_mesh()
        self.dolly_mesh = pytorch3d.io.load_objs_as_meshes(["/home/pordipatrik/UMD/II/3D Vision/hw_1/cow_on_plane.obj"])
        self.rotate_mesh=pytorch3d.io.load_objs_as_meshes([])
        
        
        

        

    def get_mesh_renderer(self, image_size=512, lights=None, device=None):
        """
        Returns a Pytorch3D Mesh Renderer.

        Args:
            image_size (int): The rendered image size.
            lights: A default Pytorch3D lights object.
            device (torch.device): The torch device to use (CPU or GPU). If not specified,
                will automatically use GPU if available, otherwise CPU.
        """
        if device is None:
            if torch.cuda.is_available():
                device = torch.device("cuda:0")
                
            else:
                device = torch.device("cpu")
                
        raster_settings = RasterizationSettings(
            image_size=image_size, blur_radius=0.0, faces_per_pixel=1,
        )
        renderer = MeshRenderer(
            rasterizer=MeshRasterizer(raster_settings=raster_settings),
            shader=HardPhongShader(device=device, lights=lights),
        )
        return renderer
    def load_mesh(self):
        vertices, faces, _ = load_obj(self.path)
        faces = faces.verts_idx
        vertices = vertices.unsqueeze(0)
        faces = faces.unsqueeze(0)
        textures = torch.ones_like(vertices)
        textures = textures * torch.tensor(self.color)
        mesh = pytorch3d.structures.Meshes(
            verts=vertices,
            faces=faces,
            textures=pytorch3d.renderer.TexturesVertex(textures)
        )
        self.mesh = mesh

        ## Color the mesh
        color_1=[1, 0, 0]
        color_2=[0.0, 0.0, 1]
        # print("c",color_1.shape)
        
        z_coordinates = vertices[:,:, 2]

        # Find the largest and smallest z-coordinates
        largest_z = torch.max(z_coordinates)
        smallest_z = torch.min(z_coordinates)

        alpha = (z_coordinates - smallest_z) / (largest_z - smallest_z)
        textures = torch.ones_like(vertices)  # (1, N_v, 3)

        color_11=textures * torch.tensor(color_1)
        color_22=textures * torch.tensor(color_2)
        
        alpha=alpha.unsqueeze(2).expand(-1, -1, 3)

        colorr = alpha * color_22 + (1 - alpha) * color_11
        
        mesh = pytorch3d.structures.Meshes(
            verts=vertices,
            faces=faces,
            textures=pytorch3d.renderer.TexturesVertex(colorr),
        )
        self.textured_mesh = mesh

    def generate_mesh(self,types="cow"):
        if types=="cow":
            mesh=self.mesh.to(self.device)
            output_file="results/my_cow_mesh.png"
        elif types=="colored":
            mesh=self.textured_mesh.to(self.device)
            output_file="results/textured_mesh.png"
        cameras = pytorch3d.renderer.FoVPerspectiveCameras(
        R=torch.eye(3).unsqueeze(0),
        T=torch.tensor([[0, 0, 3]]),
        fov=60,device=self.device)
        rend = self.renderer(mesh, device=self.device, cameras=cameras, lights=self.lights)
        rend = rend.cpu().numpy()[0, ..., :3]  # (B, H, W, 4) -> (H, W, 3)
        plt.imsave(output_file,rend)
        
    def generate_turntable_views(self, num_views=36,types="cow"):
        # Generate evenly spaced azimuth angles
        azimuths = torch.linspace(0, 360, num_views)
        if types=="cow":
            mesh=self.mesh.to(self.device)
            output_file="results/my_cow_turntable.gif"
        elif types=="dolly":
            mesh  = self.dolly_mesh.to(self.device)
            output_file="results/my_dolly_turntable.gif"
        elif types=="colored":
            mesh=self.textured_mesh.to(self.device)
            output_file="results/colored_turntable.png"
        
        # Fixed elevation and distance
        elevations = torch.tensor([30.0] * num_views)  # Fixed elevation
        distances = torch.tensor([3.0] * num_views)    # Fixed distance
        
        # Generate views using look_at_view_transform
        R, T = pytorch3d.renderer.look_at_view_transform(distances, elevations, azimuths)
        # print(R.unsqueeze(0))
        cameras = pytorch3d.renderer.FoVPerspectiveCameras(
        R=R,
        T=T,
        fov=60,
        device=self.device)
        #Render images for each view
        images=[]
        for i in cameras:
            img=(self.renderer(mesh,device=self.device, cameras=i, lights=self.lights))
            img=img.cpu().numpy()[0, ..., :3]
            images.append(img)
        
        images_pil = [Image.fromarray((img * 255).astype(np.uint8)) for img in images]
        imageio.mimsave(output_file, images_pil, duration=30, loop=0)
    
    def generate_dolly(self,image_size=256,
    num_frames=20,
    duration=3,
    device=None,
    output_file="results/my_dolly.gif",
    ):
        if device is None:
            device = torch.device("cuda:0")

        mesh  = self.dolly_mesh .to(device)
        
        renderer = get_mesh_renderer(image_size=image_size, device=device)
        lights = pytorch3d.renderer.PointLights(location=[[0.0, 0.0, -3.0]], device=device)

        fovs = torch.linspace(5, 120, num_frames)

        renders = []
        for fov in tqdm(fovs):
            distance = 5/(2*np.tan(np.radians(fov/2)))  # TODO: change this.
            # distance = 5  # TODO: change this.
            # T = [[0, 0, 2]]  # TODO: Change this.
            T = [[0, 0, distance]]
            cameras = pytorch3d.renderer.FoVPerspectiveCameras(fov=fov, T=T, device=device)
            rend = renderer(mesh, cameras=cameras, lights=lights)
            rend = rend[0, ..., :3].cpu().numpy()  # (N, H, W, 3)
            renders.append(rend)

        images = []
        for i, r in enumerate(renders):
            image = Image.fromarray((r * 255).astype(np.uint8))
            draw = ImageDraw.Draw(image)
            draw.text((20, 20), f"fov: {fovs[i]:.2f}", fill=(255, 0, 0))
            images.append(np.array(image))
        imageio.mimsave(output_file, images, duration=200, loop=0)
    
    def_rotate_cow(self):
    
        



In [81]:
cow_renderer = MyCowRenderer()

Using CUDA


In [82]:
cow_renderer.generate_mesh()
cow_renderer.generate_mesh(types="colored")
cow_renderer.generate_turntable_views(types="colored")
cow_renderer.generate_turntable_views(types="cow")
cow_renderer.generate_turntable_views(types="dolly")

In [83]:
cow_renderer.generate_dolly()

  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
 #1.
    # R_relative=[[0, 1, 0], [-1, 0, 0], [0, 0, 1]]
    # T_relative=[0, 0, 0]

    #2.
    R_relative=[[1, 0, 0], [0, 1, 0], [0, 0, 1]]
    T_relative=[0, 0, 3]

    # # #3.
    # R_relative=[[1, 0, 0], [0, 1, 0], [0, 0, 1]]
    # T_relative=[0.6, -0.5, 0]

    # #4.
    # R_relative=[[0, 0, -1], [0, 1, 0], [1, 0, 0]]
    # T_relative=[0, 0, 3]
    #torch tensornal no shift in z

    R_relative = torch.tensor(R_relative).float()
    T_relative = torch.tensor(T_relative).float()
    R = R_relative @ torch.tensor([[1.0, 0, 0], [0, 1, 0], [0, 0, 1]])
    T = R_relative @ torch.tensor([0.0, 0, 3]) + T_relative

    # # Prepare the camera:
    # cameras = pytorch3d.renderer.FoVPerspectiveCameras(
    # R=torch.eye(3).unsqueeze(0),
    # T=torch.tensor([[0, 0, 3]]),
    # fov=60,
    # device=device
    # )

     # Prepare the camera:
    cameras = pytorch3d.renderer.FoVPerspectiveCameras(
    R=R.t().unsqueeze(0),
    T=T.unsqueeze(0),
    fov=60,
    device=device
    )